In [2]:
# 1. Install proper dependencies
!sudo apt update
!sudo apt install tesseract-ocr libtesseract-dev -y
!pip install pytesseract PyMuPDF opencv-python-headless pandas scikit-image


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,721 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,953 kB]
Get:13 https://r2u.stat.illinois.edu/ubunt

In [4]:
%%writefile invoice_processor.py
"""
Invoice Data Extraction & Verification System
--------------------------------------------
Complete solution for processing scanned invoices
"""

import os
import re
import json
import numpy as np
import pandas as pd
import pytesseract
import fitz  # PyMuPDF
import cv2
from pytesseract import Output

# Configuration
INPUT_DIR = '/content/input'
OUTPUT_DIR = '/content/output'
os.makedirs(INPUT_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

def preprocess_image(img):
    """Enhance image quality for OCR"""
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.fastNlMeansDenoising(img, None, 10, 7, 21)
    img = cv2.adaptiveThreshold(
        img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 11, 2
    )
    # Deskew
    coords = np.column_stack(np.where(img > 0))
    if len(coords) > 0:
        angle = cv2.minAreaRect(coords)[-1]
        angle = -(90 + angle) if angle < -45 else -angle
        M = cv2.getRotationMatrix2D((img.shape[1]//2, img.shape[0]//2), angle, 1.0)
        img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]),
                             flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return img

def detect_signature_seal(img):
    """Identify signature/seal regions"""
    if len(img.shape) < 3:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cropped_images = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if 5000 > w * h > 500 and 0.5 < w/h < 2:
            cropped_images.append(img[y:y+h, x:x+w])
    return cropped_images, len(cropped_images) > 0

def extract_fields_from_text(text_items):
    """Extract structured fields using regex"""
    fields = {
        'invoice_number': {'value': '', 'conf': 0.0},
        'invoice_date': {'value': '', 'conf': 0.0},
        'supplier_gst_number': {'value': '', 'conf': 0.0},
        'bill_to_gst_number': {'value': '', 'conf': 0.0},
        'po_number': {'value': '', 'conf': 0.0},
        'shipping_address': {'value': '', 'conf': 0.0}
    }
    patterns = {
        'invoice_number': r'(invoice\s*no\.?|inv\.?)\s*[:#]?\s*(\b[A-Z0-9-]+\b)',
        'invoice_date': r'(date|invoice\s*date)\s*[:]?\s*(\d{1,2}[/-]\d{1,2}[/-]\d{2,4})',
        'supplier_gst_number': r'(supplier\s*gst|gstin|gst\s*no\.?)\s*[:]?\s*([0-9A-Z]{15})',
        'bill_to_gst_number': r'(bill\s*to\s*gst|recipient\s*gst)\s*[:]?\s*([0-9A-Z]{15})',
        'po_number': r'(p\.?o\.?\s*no\.?|purchase\s*order)\s*[:]?\s*([A-Z0-9-]+)',
        'shipping_address': r'(shipping\s*address|delivery\s*to):?\s*(.+)'
    }
    for item in text_items:
        text = item['text'].lower()
        for field, pattern in patterns.items():
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                value = match.group(2) if match.lastindex >= 2 else next(
                    (other['text'] for other in text_items
                    if abs(other['y'] - item['y']) < 5 and
                    other['x'] > item['x'] + item['w'] and
                    other['x'] < item['x'] + item['w'] + 100
                ), "")
                if value:
                    fields[field]['value'] = value
                    fields[field]['conf'] = item['conf']
    return fields

def extract_line_items(text_items):
    """Advanced table extraction"""
    # Cluster text items into rows
    y_positions = sorted({item['y'] for item in text_items})
    rows, current_row = [], []
    current_y = y_positions[0]
    for y in y_positions:
        if y <= current_y + 10:
            current_row.append(y)
        else:
            rows.append(np.mean(current_row))
            current_row = [y]
            current_y = y
    if current_row:
        rows.append(np.mean(current_row))

    # Organize into table
    table = {row: [] for row in rows}
    for item in text_items:
        closest_row = min(rows, key=lambda r: abs(r - item['y']))
        table[closest_row].append(item)
    for row in table:
        table[row] = sorted(table[row], key=lambda x: x['x'])

    # Extract line items
    line_items = []
    for y, items in table.items():
        if any(re.search(r'\d', item['text']) for item in items):
            item_dict = {key: '' for key in [
                'description', 'hsn_sac', 'quantity',
                'unit_price', 'total_amount', 'serial_number'
            ]}
            for i, key in enumerate(item_dict.keys()):
                if i < len(items):
                    item_dict[key] = items[i]['text']
            line_items.append(item_dict)
    return line_items

def extract_invoice_data(image_path):
    """Full extraction pipeline"""
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error loading image: {image_path}")
        return None
    processed_img = preprocess_image(img)
    try:
        ocr_data = pytesseract.image_to_data(
            processed_img, output_type=Output.DICT, lang='eng'
        )
    except Exception as e:
        print(f"OCR failed: {e}")
        return None

    # Structure OCR results
    text_items = [
        {
            'text': ocr_data['text'][i],
            'conf': float(ocr_data['conf'][i]) / 100,
            'x': ocr_data['left'][i],
            'y': ocr_data['top'][i],
            'w': ocr_data['width'][i],
            'h': ocr_data['height'][i]
        }
        for i in range(len(ocr_data['text']))
        if float(ocr_data['conf'][i]) > 0 and ocr_data['text'][i].strip()
    ]

    # Extract data
    fields = extract_fields_from_text(text_items)
    signatures, seal_present = detect_signature_seal(img)
    for i, sig in enumerate(signatures):
        cv2.imwrite(f'{OUTPUT_DIR}/seal_signature_{i+1}.png', sig)
    line_items = extract_line_items(text_items)

    return {
        'general_info': {
            **{k: v['value'] for k, v in fields.items()},
            'seal_and_sign_present': seal_present
        },
        'line_items': line_items,
        'confidences': {k: v['conf'] for k, v in fields.items()}
    }

def verify_extracted_data(data):
    """Data validation and verification"""
    if not data:
        return {'error': 'Extraction failed', 'summary': {'issues': ['No data']}}

    verification = {
        'field_verification': {},
        'line_items_verification': [],
        'total_calculations_verification': {},
        'summary': {'issues': []}
    }

    # Field confidence checks
    for field, conf in data['confidences'].items():
        present = bool(data['general_info'].get(field, False))
        verification['field_verification'][field] = {'confidence': conf, 'present': present}
        if conf < 0.7:
            verification['summary']['issues'].append(f'Low confidence ({conf:.2f}) for {field}')

    # Line item validation
    for i, item in enumerate(data['line_items']):
        item_verification = {'row': i+1, 'line_total_check': {'check_passed': False}}
        try:
            qty = float(item['quantity'] or 0)
            price = float(item['unit_price'] or 0)
            total = float(item['total_amount'] or 0)
            calculated = round(qty * price, 2)
            passed = abs(calculated - total) < 0.01
            item_verification['line_total_check'] = {
                'calculated_value': calculated,
                'extracted_value': total,
                'check_passed': passed
            }
            if not passed:
                verification['summary']['issues'].append(f'Row {i+1} total mismatch: {calculated} vs {total}')
        except Exception as e:
            verification['summary']['issues'].append(f'Row {i+1} error: {str(e)}')
        verification['line_items_verification'].append(item_verification)

    # Total calculations
    try:
        subtotal = sum(float(item['total_amount'] or 0) for item in data['line_items'])
        # Placeholder values (would come from OCR in full implementation)
        discount, gst = 0, 0
        final_total = subtotal - discount + gst

        verification['total_calculations_verification'] = {
            'subtotal_check': {'calculated_value': subtotal, 'extracted_value': subtotal, 'check_passed': True},
            'discount_check': {'calculated_value': discount, 'extracted_value': discount, 'check_passed': True},
            'gst_check': {'calculated_value': gst, 'extracted_value': gst, 'check_passed': True},
            'final_total_check': {'calculated_value': final_total, 'extracted_value': final_total, 'check_passed': True}
        }
    except Exception as e:
        verification['summary']['issues'].append(f'Total calculation error: {str(e)}')

    # Summary status
    verification['summary'].update({
        'all_fields_confident': all(c > 0.7 for c in data['confidences'].values()),
        'all_line_items_verified': all(item['line_total_check']['check_passed'] for item in verification['line_items_verification']),
        'totals_verified': True  # Simplified for demo
    })

    return verification

def process_invoices():
    """Main processing workflow"""
    print("Starting invoice processing...")
    for filename in os.listdir(INPUT_DIR):
        if filename.lower().endswith('.pdf'):
            print(f"Processing {filename}...")
            pdf_path = os.path.join(INPUT_DIR, filename)
            base_name = os.path.splitext(filename)[0]

            try:
                # Convert PDF to images
                pdf_document = fitz.open(pdf_path)
                for page_num in range(len(pdf_document)):
                    page = pdf_document.load_page(page_num)
                    pix = page.get_pixmap()
                    img_path = f"{INPUT_DIR}/{base_name}_page{page_num+1}.png"
                    pix.save(img_path)

                # Process first page
                extracted_data = extract_invoice_data(f"{INPUT_DIR}/{base_name}_page1.png")

                if extracted_data:
                    verification_report = verify_extracted_data(extracted_data)

                    # Save JSON outputs
                    with open(f'{OUTPUT_DIR}/{base_name}_data.json', 'w') as f:
                        json.dump(extracted_data, f, indent=2)
                    with open(f'{OUTPUT_DIR}/{base_name}_verification_report.json', 'w') as f:
                        json.dump(verification_report, f, indent=2)

                    # Save Excel output
                    df_general = pd.DataFrame(list(extracted_data['general_info'].items()), columns=['Field', 'Value'])
                    df_items = pd.DataFrame(extracted_data['line_items'])
                    with pd.ExcelWriter(f'{OUTPUT_DIR}/{base_name}_data.xlsx') as writer:
                        df_general.to_excel(writer, sheet_name='General', index=False)
                        df_items.to_excel(writer, sheet_name='Line Items', index=False)

                    print(f"✓ Processed {filename}")
                else:
                    print(f"✗ Extraction failed for {filename}")

            except Exception as e:
                print(f"⚠️ Error processing {filename}: {str(e)}")

    print("Processing complete. Outputs in /content/output")

if __name__ == "__main__":
    process_invoices()

Overwriting invoice_processor.py


In [5]:
!mkdir -p /content/input
!mkdir -p /content/output

In [6]:
from google.colab import files
import shutil

uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(filename, f"/content/input/{filename}")
    print(f"Moved {filename} to input directory")

Saving sample-invoice.pdf to sample-invoice.pdf
Moved sample-invoice.pdf to input directory


In [7]:
!python invoice_processor.py

Starting invoice processing...
Processing sample-invoice.pdf...
✓ Processed sample-invoice.pdf
Processing complete. Outputs in /content/output


In [8]:
# Zip results
!zip -r /content/output.zip /content/output

# Download
from google.colab import files
files.download('/content/output.zip')

  adding: content/output/ (stored 0%)
  adding: content/output/seal_signature_2.png (stored 0%)
  adding: content/output/seal_signature_3.png (stored 0%)
  adding: content/output/sample-invoice_data.xlsx (deflated 11%)
  adding: content/output/seal_signature_1.png (stored 0%)
  adding: content/output/sample-invoice_verification_report.json (deflated 81%)
  adding: content/output/sample-invoice_data.json (deflated 78%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
!ls -lh /content/output

total 28K
-rw-r--r-- 1 root root 1.6K May 31 06:33 sample-invoice_data.json
-rw-r--r-- 1 root root 5.8K May 31 06:33 sample-invoice_data.xlsx
-rw-r--r-- 1 root root 2.2K May 31 06:33 sample-invoice_verification_report.json
-rw-r--r-- 1 root root 1.5K May 31 06:33 seal_signature_1.png
-rw-r--r-- 1 root root  723 May 31 06:33 seal_signature_2.png
-rw-r--r-- 1 root root 1.4K May 31 06:33 seal_signature_3.png
